<a href="https://colab.research.google.com/github/ruhullahil/gcolab/blob/master/Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from google.colab import files

In [0]:
uploaded = files.upload()

data = open("Othello.txt").read().lower()
#This get the set of characters used in the data and sorts them
print(data)
chars = sorted(list(set(data)))
#Total number of characters used in the data
totalChars = len(data)
print(totalChars)
#Number of unique chars
numberOfUniqueChars = len(chars)

Saving Othello.txt to Othello (3).txt
her father loved me, oft invited me;
still questioned me the story of my life
from year to year -- the battles, sieges, fortunes
that i have passed.
i ran it through, even from my boyish days
to th' very moment that he bade me tell it.
wherein i spoke of most disastrous chances,
of moving accidents by flood and field;
of hairbreadth scapes i' the' imminent deadly breach;
of being taken by the insolent foe
and sold to slavery; of my redemption thence
and portance in my travels' history;
wherein of anters vast and deserts idle,
rough quarries, rocks, and hills whose heads touch heaven,
it was my hint to speak -- such was the process;
and of the cannibals that each other eat,
the anthropophagi, and men whose heads
do grow beneath their shoulders. this to hear
would desdemona seriously incline;
but still the house affairs would draw her thence;
which ever she could with haste dispatch,
she'd come again, and with a greedy ear
devour up my discourse. whi

In [0]:

#This allows for characters to be represented by numbers
CharsForids = {char:Id for Id, char in enumerate(chars)}

#This is the opposite to the above
idsForChars = {Id:char for Id, char in enumerate(chars)}

#How many timesteps e.g how many characters we want to process in one go
numberOfCharsToLearn = 100

#Since our timestep sequence represetns a process for every 100 chars we omit
#the first 100 chars so the loop runs a 100 less or there will be index out of
#range
counter = totalChars - numberOfCharsToLearn

#Inpput data
charX = []
#output data
y = []
#This loops through all the characters in the data skipping the first 100
for i in range(0, counter, 1):
    #This one goes from 0-100 so it gets 100 values starting from 0 and stops
    #just before the 100th value
    theInputChars = data[i:i+numberOfCharsToLearn]
    #With no : you start with 0, and so you get the actual 100th value
    #Essentially, the output Chars is the next char in line for those 100 chars
    #in X
    theOutputChars = data[i + numberOfCharsToLearn]
    #Appends every 100 chars ids as a list into X
    charX.append([CharsForids[char] for char in theInputChars])
    #For every 100 values there is one y value which is the output
    y.append(CharsForids[theOutputChars])

#Len charX represents how many of those time steps we have
#Our features are set to 1 because in the output we are only predicting 1 char
#Finally numberOfCharsToLearn is how many character we process
X = np.reshape(charX, (len(charX), numberOfCharsToLearn, 1))

#This is done for normalization
X = X/float(numberOfUniqueChars)

#This sets it up for us so we can have a categorical(#feature) output format
y = np_utils.to_categorical(y)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:

model = Sequential()
#Since we know the shape of our Data we can input the timestep and feature data
#The number of timestep sequence are dealt with in the fit function
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
#number of features on the output
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=5, batch_size=128)
model.save_weights("Othello.hdf5")
#model.load_weights("Othello.hdf5")

randomVal = np.random.randint(0, len(charX)-1)
randomStart = charX[randomVal]
for i in range(500):
    x = np.reshape(randomStart, (1, len(randomStart), 1))
    x = x/float(numberOfUniqueChars)
    pred = model.predict(x)
    index = np.argmax(pred)
    randomStart.append(index)
    randomStart = randomStart[1: len(randomStart)]
print("".join([idsForChars[value] for value in randomStart]))

Epoch 1/5
1760/1760 [==============================] - 2s 1ms/step - loss: 3.2352
Epoch 2/5
1760/1760 [==============================] - 2s 1ms/step - loss: 3.0390
Epoch 3/5
1760/1760 [==============================] - 2s 1ms/step - loss: 3.0007
Epoch 4/5
1760/1760 [==============================] - 2s 1ms/step - loss: 2.9951
Epoch 5/5
1760/1760 [==============================] - 2s 1ms/step - loss: 2.9891
                                                                                                    
